<a href="https://colab.research.google.com/github/ThisIsFarhan/DocumentSorter-BERTopic/blob/main/Doc_Sorter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic datasets ctransformers[cuda] --upgrade git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2vnx38v1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-2vnx38v1
  Resolved https://github.com/huggingface/transformers to commit dcbdf7e962c4b36140cc9ee76f870016121e69e5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 M

In [6]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from bertopic.representation import TextGeneration
from bertopic import BERTopic

In [3]:
dataset = load_dataset("maartengr/arxiv_nlp")["train"]
abstracts = dataset["Abstracts"]
abstracts

README.md:   0%|          | 0.00/617 [00:00<?, ?B/s]

data.csv:   0%|          | 0.00/53.2M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

['  In this paper Arabic was investigated from the speech recognition problem\npoint of view. We propose a novel approach to build an Arabic Automated Speech\nRecognition System (ASR). This system is based on the open source CMU Sphinx-4,\nfrom the Carnegie Mellon University. CMU Sphinx is a large-vocabulary;\nspeaker-independent, continuous speech recognition system based on discrete\nHidden Markov Models (HMMs). We build a model using utilities from the\nOpenSource CMU Sphinx. We will demonstrate the possible adaptability of this\nsystem to Arabic voice recognition.\n',
 '  In this paper we present the creation of an Arabic version of Automated\nSpeech Recognition System (ASR). This system is based on the open source\nSphinx-4, from the Carnegie Mellon University. Which is a speech recognition\nsystem based on discrete hidden Markov models (HMMs). We investigate the\nchanges that must be made to the model to adapt Arabic voice recognition.\n  Keywords: Speech recognition, Acoustic mo

In [4]:
embedding_model = SentenceTransformer("thenlper/gte-small")
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)
umap_model = UMAP(
                n_components=5, min_dist=0.0, metric='cosine', random_state=42
              )
hdbscan_model = HDBSCAN(
    min_cluster_size = 50,
    metric = 'euclidean',
    cluster_selection_method='eom',
    prediction_data=True
)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1405 [00:00<?, ?it/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/zephyr-7B-alpha-GGUF",
    model_file="zephyr-7b-alpha.Q4_K_M.gguf",
    model_type="mistral",
    gpu_layers=50,
    hf=True
)

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
generator = pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    max_new_tokens=50,
    repetition_penalty=1.1
)

prompt = """<|system|>You are a helpful, respectful and honest assistant for labeling topics..</s>
<|user|>
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the information about the topic above, please create a short label of this topic. Make sure you to only return the label and nothing more.</s>
<|assistant|>"""


zephyr = TextGeneration(generator, prompt=prompt)
representation_model = {"Zephyr": zephyr}

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

zephyr-7b-alpha.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
prompt = """I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords: '[KEYWORDS]'.

Based on the documents and keywords, what is this topic about?"""

# Update our topic representations using Flan-T5
generator = pipeline('text2text-generation', model='google/flan-t5-small')
representation_model = TextGeneration(
    generator, prompt=prompt, doc_length=50, tokenizer="whitespace"
)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
    verbose=True
)
topics, probs = topic_model.fit_transform(abstracts, embeddings)

2025-03-03 12:29:20,251 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-03-03 12:30:22,988 - BERTopic - Dimensionality - Completed ✓
2025-03-03 12:30:22,991 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-03-03 12:30:30,450 - BERTopic - Cluster - Completed ✓
2025-03-03 12:30:30,463 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 153/153 [00:13<00:00, 11.03it/s]
2025-03-03 12:30:52,484 - BERTopic - Representation - Completed ✓


In [15]:
topic_model.get_topic(4, full=True)

{'Main': [('Science/Tech', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)]}

In [14]:
topic_model.get_topics()

{-1: [('Word representation', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)],
 0: [('Question answering systems', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)],
 1: [('Speech recognition', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)],
 2: [('Image-language learning', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)],
 3: [('Summarization', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)],
 4: [('Science/Tech', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)],
 5: [('Hate Speech', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)],
 6: [('Science/Tech', 1),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0),
  ('', 0)

In [16]:
topic_info = topic_model.get_topic_info()
print(topic_info)


     Topic  Count                             Name  \
0       -1  14462        -1_Word representation___   
1        0   2241  0_Question answering systems___   
2        1   2098          1_Speech recognition___   
3        2    903     2_Image-language learning___   
4        3    887               3_Summarization___   
..     ...    ...                              ...   
148    147     54               147_Psychiatric___   
149    148     53              148_Science/Tech___   
150    149     52               149_Code-mixing___   
151    150     51                 150_Diffusion___   
152    151     51                 151_Coherence___   

                                     Representation  \
0           [Word representation, , , , , , , , , ]   
1    [Question answering systems, , , , , , , , , ]   
2            [Speech recognition, , , , , , , , , ]   
3       [Image-language learning, , , , , , , , , ]   
4                 [Summarization, , , , , , , , , ]   
..                   

In [18]:
new_abstract = ["In recent years, Question Answering (QA) systems have gained significant attention due to their ability to provide precise answers to user queries. These systems leverage natural language processing (NLP) and deep learning models to extract relevant information from structured and unstructured data sources. Modern QA architectures, such as retrieval-based and generative models, enhance response accuracy by integrating transformer-based models like BERT, GPT, and T5. Challenges such as context understanding, ambiguity resolution, and domain adaptation remain crucial for improving performance. This study explores advancements in QA systems, emphasizing knowledge graph integration, fine-tuning strategies, and real-world applications in customer support, education, and healthcare."]

topic, prob = topic_model.transform(new_abstract)
print(f"Predicted Topic: {topic[0]}, Probability: {prob[0]}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-03 12:38:14,009 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-03-03 12:38:14,022 - BERTopic - Dimensionality - Completed ✓
2025-03-03 12:38:14,024 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-03-03 12:38:14,027 - BERTopic - Cluster - Completed ✓


Predicted Topic: 0, Probability: 1.0


In [19]:
topic_model.get_topic(0)

[('Question answering systems', 1),
 ('', 0),
 ('', 0),
 ('', 0),
 ('', 0),
 ('', 0),
 ('', 0),
 ('', 0),
 ('', 0),
 ('', 0)]

In [20]:
topic_model.get_representative_docs(148)


['  Over the last few years, large language models (LLMs) have emerged as the\nmost important breakthroughs in natural language processing (NLP) that\nfundamentally transform research and developments in the field. ChatGPT\nrepresents one of the most exciting LLM systems developed recently to showcase\nimpressive skills for language generation and highly attract public attention.\nAmong various exciting applications discovered for ChatGPT in English, the\nmodel can process and generate texts for multiple languages due to its\nmultilingual training data. Given the broad adoption of ChatGPT for English in\ndifferent problems and areas, a natural question is whether ChatGPT can also be\napplied effectively for other languages or it is necessary to develop more\nlanguage-specific technologies. The answer to this question requires a thorough\nevaluation of ChatGPT over multiple tasks with diverse languages and large\ndatasets (i.e., beyond reported anecdotes), which is still missing or limi

In [23]:
topics, probs = topic_model.transform(["  Over the last few years, large language models (LLMs) have emerged as the\nmost important breakthroughs in natural language processing (NLP) that\nfundamentally transform research and developments in the field. ChatGPT\nrepresents one of the most exciting LLM systems developed recently to showcase\nimpressive skills for language generation and highly attract public attention.\nAmong various exciting applications discovered for ChatGPT in English, the\nmodel can process and generate texts for multiple languages due to its\nmultilingual training data. Given the broad adoption of ChatGPT for English in\ndifferent problems and areas, a natural question is whether ChatGPT can also be\napplied effectively for other languages or it is necessary to develop more\nlanguage-specific technologies. The answer to this question requires a thorough\nevaluation of ChatGPT over multiple tasks with diverse languages and large\ndatasets (i.e., beyond reported anecdotes), which is still missing or limited\nin current research. Our work aims to fill this gap for the evaluation of\nChatGPT and similar LLMs to provide more comprehensive information for\nmultilingual NLP applications. While this work will be an ongoing effort to\ninclude additional experiments in the future, our current paper evaluates\nChatGPT on 7 different tasks, covering 37 diverse languages with high, medium,\nlow, and extremely low resources. We also focus on the zero-shot learning\nsetting for ChatGPT to improve reproducibility and better simulate the\ninteractions of general users. Compared to the performance of previous models,\nour extensive experimental results demonstrate a worse performance of ChatGPT\nfor different NLP tasks and languages, calling for further research to develop\nbetter models and understanding for multilingual learning.\n"])
print(f"Predicted Topic: {topics[0]}, Probability: {probs[0]}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-03 12:42:03,801 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-03-03 12:42:03,809 - BERTopic - Dimensionality - Completed ✓
2025-03-03 12:42:03,811 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-03-03 12:42:03,815 - BERTopic - Cluster - Completed ✓


Predicted Topic: 148, Probability: 1.0


In [24]:
topic_model.save("bertopic_model")
# loaded_model = BERTopic.load("bertopic_model")

2025-03-03 12:55:28,898 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [26]:
embedding_model = "thenlper/gte-small"
topic_model.save("my_model_dir", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

In [27]:
from bertopic import BERTopic
embedding_model1 = "thenlper/gte-small"
loaded_model = BERTopic.load("my_model_dir", embedding_model=embedding_model1)


In [30]:
topics, probs = loaded_model.transform(["  Over the last few years, large language models (LLMs) have emerged as the\nmost important breakthroughs in natural language processing (NLP) that\nfundamentally transform research and developments in the field. ChatGPT\nrepresents one of the most exciting LLM systems developed recently to showcase\nimpressive skills for language generation and highly attract public attention.\nAmong various exciting applications discovered for ChatGPT in English, the\nmodel can process and generate texts for multiple languages due to its\nmultilingual training data. Given the broad adoption of ChatGPT for English in\ndifferent problems and areas, a natural question is whether ChatGPT can also be\napplied effectively for other languages or it is necessary to develop more\nlanguage-specific technologies. The answer to this question requires a thorough\nevaluation of ChatGPT over multiple tasks with diverse languages and large\ndatasets (i.e., beyond reported anecdotes), which is still missing or limited\nin current research. Our work aims to fill this gap for the evaluation of\nChatGPT and similar LLMs to provide more comprehensive information for\nmultilingual NLP applications. While this work will be an ongoing effort to\ninclude additional experiments in the future, our current paper evaluates\nChatGPT on 7 different tasks, covering 37 diverse languages with high, medium,\nlow, and extremely low resources. We also focus on the zero-shot learning\nsetting for ChatGPT to improve reproducibility and better simulate the\ninteractions of general users. Compared to the performance of previous models,\nour extensive experimental results demonstrate a worse performance of ChatGPT\nfor different NLP tasks and languages, calling for further research to develop\nbetter models and understanding for multilingual learning.\n"])
print(f"Predicted Topic: {topics[0]}, Probability: {probs[0]}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-03-03 13:00:07,071 - BERTopic - Predicting topic assignments through cosine similarity of topic and document embeddings.


Predicted Topic: 148, Probability: 0.9579033255577087


In [29]:
import shutil
shutil.make_archive('my_model_dir', 'zip', 'my_model_dir')

'/content/my_model_dir.zip'